In [1]:
from database.mainstbets import MainStBets
import pandas as pd
from tqdm import tqdm
from datetime import datetime,timedelta
import matplotlib.pyplot as plt

In [16]:
market = MainStBets()

In [17]:
market.connect()
sp500 = market.retrieve("sp500")
prices = market.retrieve("prices")
market.disconnect()

In [18]:
sp500.columns

Index(['_id', 'Symbol', 'Security', 'SEC filings', 'GICS Sector',
       'GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK',
       'Founded'],
      dtype='object')

In [19]:
prices["date"] = pd.to_datetime(prices["date"])

In [20]:
prices.columns

Index(['_id', 'date', 'close', 'high', 'low', 'open', 'volume', 'adjClose',
       'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitFactor',
       'ticker'],
      dtype='object')

In [21]:
final = []
for ticker in tqdm(sp500["Symbol"]):
    ticker_data = prices[prices["ticker"]==ticker]
    ticker_data["rolling"] = ticker_data["adjClose"].rolling(window=100).mean()
    ticker_data["gain"] = (ticker_data["adjClose"] - ticker_data["rolling"]) / ticker_data["rolling"]
    final.append(ticker_data)

  0%|                                                                                         | 0/505 [00:00<?, ?it/s]C:\Users\ericj\AppData\Local\Temp/ipykernel_12052/1279899860.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_data["rolling"] = ticker_data["adjClose"].rolling(window=100).mean()
C:\Users\ericj\AppData\Local\Temp/ipykernel_12052/1279899860.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_data["gain"] = (ticker_data["adjClose"] - ticker_data["rolling"]) / ticker_data["roll

In [8]:
sim = pd.concat(final)
sim.dropna(inplace=True)

In [9]:
sim["year"] = [x.year for x in sim["date"]]

In [10]:
today = sim[sim["date"]==sim["date"].max()][["date","adjClose","ticker","gain","rolling"]].sort_values("gain").merge(sp500[["Symbol","Security","GICS Sector"]].rename(columns={"Symbol":"ticker"}),on="ticker")

In [11]:
recent = today[["date","GICS Sector","Security","ticker","rolling","adjClose","gain"]]

In [12]:
recent

,date,GICS Sector,Security,ticker,rolling,adjClose,gain


In [13]:
full = sim[["date","adjClose","ticker","gain","rolling"]].merge(sp500[["Symbol","Security","GICS Sector"]].rename(columns={"Symbol":"ticker"}),on="ticker")[["date","GICS Sector","Security","ticker","rolling","adjClose","gain"]]

In [14]:
recent.index.size

0

In [15]:
market.connect()
market.store("full",full)
market.store("recent",recent)
market.disconnect()

mainstbets full documents must be a non-empty list
mainstbets recent documents must be a non-empty list
